# Домашня робота №3
Будемо використовувати *Збірник задач з теорії ймовірностей та математичної статистики: навч. посібник / В.В. Голомозий, М.В. Карташов, К.В. Ральченко. – К.: Видавничо-поліграфічний центр «Київський університет», 2015. – 366 с.*
Електронну версію збірника можна знайти [тут](http://probability.univ.kiev.ua/userfiles/kmv/gkr-problems.pdf).

In [35]:
from math import gcd
from numpy import log
from numpy.random import choice, shuffle
from itertools import product, combinations_with_replacement, combinations

## 1 Задача 1.3.12
Знайти ймовiрнiсть того, що серед трьох цифр, кожна з яких вибрана навмання, буде лише 1, 2, 3 рiзних.

Обчисліть теоретичну (повним перебором) та еміричну (симулюванням $100000$ експериментів) імовірності.

In [36]:
def theoretic(tests=100000):
    prob = [0, 0, 0]
    for _ in range(tests):
        digit = choice(range(10), size=3)
        prob[3 - len(set(digit))] += 1

    return tuple(map(lambda x: x/tests, prob))


def empiric():
    prob = [0, 0, 0]
    digits = product(*[range(10)]*3)
    for i in digits: prob[3 - len(set(i))] += 1

    return tuple(map(lambda x: x/10**3, prob))


print(f'Theoretic: {tuple(map(lambda x: round(x, 2), theoretic()))}')
print(f'Empiric: {empiric()}')

Theoretic: (0.72, 0.27, 0.01)
Empiric: (0.72, 0.27, 0.01)


# 2 Задача 1.3.14
З послiдовностi чисел $1, 2, . . . , n$ вибирають навмання $k$ рiзних чисел.
Яка ймовiрнiсть того, що:
1. кожне з вибраних чисел кратне даному числу $p$;
2. кожне з вибраних чисел кратне хоча б одному з двох взаємно простих чисел $p$ i $q$;
3. серед вибраних чисел є хоча б одне кратне $p$?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k, p, q$.
Виведіть результат для
- $n = 25, k = 5, p = 3, q = 4$;
- $n = 25, k = 10, p = 3, q = 4$.

In [37]:
def coprime(a, b):
    return gcd(a, b) == 1


def theoretic(n, k, p, q):
    
    def check(num):
        return len(set(num)) == len(num)
    
    prob = [0, 0, 0]
    digits = combinations_with_replacement(range(1, n + 1), k)
    c = 0

    for i in digits:
        if not check(i): continue

        c += 1
        success0 = 1 
        success1 = 1 
        success2 = 0 

        for digit in i:
            if digit % p:
                success0 = 0
            if not digit % p:
                success2 = 1
            if digit % p and digit % q:
                success1 = 0

        if success0:
            prob[0] += 1
        if success1:
            prob[1] += 1
        if success2:
            prob[2] += 1

    if not coprime(p, q):
        prob[1] = 0

    return tuple(map(lambda x: x/c, prob))


def empiric(n, k, p, q, test=100000):
    prob = [0, 0, 0]
    for _ in range(test):
        digits = choice(range(1, n + 1), k)
        success0 = 1
        success1 = 1 
        success2 = 0 
        
        for digit in digits:
            if digit % p:
                success0 = 0
            if not digit % p:
                success2 = 1
            if digit % p and digit % q:
                success1 = 0

        if success0:
            prob[0] += 1
        if success1:
            prob[1] += 1
        if success2:
            prob[2] += 1

    if not coprime(p, q):
        prob[1] = 0

    return tuple(map(lambda x: x/test, prob))


print(f'Theoretic 1: {tuple(map(lambda x: round(x, 2), theoretic(25, 5, 3, 4)))}')
print(f'Empiric 1: {empiric(25, 5, 3, 4)}\n')

print(f'Theoretic 2: {tuple(map(lambda x: round(x, 2), theoretic(25, 10, 3, 4)))}')
print(f'Empiric 2: {empiric(25, 10, 3, 4)}')

Theoretic 1: (0.0, 0.01, 0.88)
Empiric 1: (0.00325, 0.02568, 0.85503)

Theoretic 2: (0.0, 0.0, 0.99)
Empiric 2: (0.0, 0.00068, 0.97953)


# 3 Задача 1.4.6
В коморi знаходяться $n$ пар черевикiв.
З них випадковим чином вибираються $2k$ черевикiв.
Яка ймовiрнiсть того, що серед вибраних черевикiв:
1. вiдсутнi парнi;
2. є рiвно одна комплектна пара;
3. є двi комплектнi пари?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k$.
Виведіть результат для
- $n = 8, k = 4$;
- $n = 13, k = 5$.

In [38]:
alphabet = "abcdefghijklmnopqrstuvwxyz"


def theoretic(n, k):
    cur = alphabet[:n]
    pairs = [(i, 1) for i in cur] + [(i, 2) for i in cur]
    prob = [0, 0, 0]
    c = 0

    for comb in combinations(pairs, 2 * k):
        c += 1
        success1 = 1
        tmp = {}
        for a in cur:
            tmp[a] = 0
        for pair in comb:
            tmp[pair[0]] += 1
            
        num_pairs = len([i for i in tmp.values() if i == 2])
        if num_pairs == 0:
            prob[0] += 1
        if num_pairs == 1:
            prob[1] += 1
        if num_pairs == 2:
            prob[2] += 1

    return tuple(map(lambda x: x/c, prob))


def empiric(n, k, test=100000):
    cur = alphabet[:n]
    pairs = [(i, 1) for i in cur] + [(i, 2) for i in cur]
    prob = [0, 0, 0]

    for _ in range(test):
        temp = pairs[:]
        shuffle(temp)
        comb = temp[:2 * k]
        success1 = 1
        tmp = {}

        for a in cur:
            tmp[a] = 0
        for pair in comb:
            tmp[pair[0]] += 1

        num_pairs = len([i for i in tmp.values() if i == 2])
        if num_pairs == 0:
            prob[0] += 1
        if num_pairs == 1:
            prob[1] += 1
        if num_pairs == 2:
            prob[2] += 1

    return tuple(map(lambda x: x/test, prob))


print(f'Theoretic 1: {tuple(map(lambda x: round(x, 2), theoretic(8, 4)))}')
print(f'Empiric 1: {tuple(map(lambda x: round(x, 2), empiric(8, 4)))}\n')

print(f'Theoretic 2: {tuple(map(lambda x: round(x, 2), theoretic(13, 5)))}')
print(f'Empiric 2: {tuple(map(lambda x: round(x, 2), empiric(13, 5)))}')

Theoretic 1: (0.02, 0.28, 0.52)
Empiric 1: (0.02, 0.28, 0.52)

Theoretic 2: (0.06, 0.31, 0.43)
Empiric 2: (0.05, 0.31, 0.43)


# 4 Задача 1.2.15
Нехай $\Omega = {1, 2, \ldots, 2n}$.
Всiм числам приписанi ймовiрностi, пропорцiйнi логарифмам цих чисел.
Знайти цi ймовiрностi.
Знайти ймовiрнiсть того, що в результатi експерименту з’явиться:
1. парне число;
2. непарне число.

Напишіть функцію для обрахунку еміричної (симулюванням $100000$ експериментів) імовірності в залежності від параметра $n$.
Виведіть результат для
- $n = 10$;
- $n = 25$.

In [39]:
def empiric(n):
    coef = 1/sum([log(i) for i in range(1, 2*n + 1)])
    prob = [coef*log(i) for i in range(1, 2*n + 1)]
    num = [0, 0]
    
    for _ in range(100000):
        tmp = choice(range(1, 2*n + 1), p=prob)
        num[tmp % 2] += 1
    return coef, prob, tuple(map(lambda x: x/100000, num))


n = (10, 25)

for i in n:
    coef, prob, num = empiric(i)
    print(f'n = {i}:')
    print(f'probabillties:\n{tuple(map(lambda x: round(x, 2), prob))}')
    print(f'num: {num}\n')

n = 10:
probabillties:
(0.0, 0.02, 0.03, 0.03, 0.04, 0.04, 0.05, 0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07, 0.07, 0.07, 0.07, 0.07)
num: (0.52202, 0.47798)

n = 25:
probabillties:
(0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03)
num: (0.50763, 0.49237)

